# Analyis of Fast Folding Trajectories using TICA and molPX
<pre>
 Guillermo Pérez Hernández guille.perez@fu-berlin.de
 Simon Olsson solsson@zedat.fu-berlin.de
</pre>

The data used was generated by DE Shaw Research on the Anton Supercomputer and kindly made available by their lab. The original work is
    
    Kresten Lindorff-Larsen, Stefano Piana, Ron O. Dror, David E. Shaw
    Science  28 Oct 2011, 
    Vol. 334, Issue 6055, pp. 517-520
    DOI: 10.1126/science.1208351


In [ ]:
import molpx
import numpy as np
import matplotlib.pylab as plt
from glob import glob
from ipywidgets import Text
from IPython.display import display
import os
%matplotlib notebook

## Available projects

In [ ]:
basedir = '/home/mi/gph82/work/CMB/benchmarks/'
codes = open(os.path.join(basedir, 'project_list.dat')).read().splitlines()
for code in codes:
    print(code)

## Simon's TICA projects

In [ ]:
idx = 0
available_fnames = []
for code in codes:
    ilist = molpx._bmutils._sort_nicely(glob(os.path.join(basedir, code+'/*npz')))
    available_fnames += ilist
    for l in ilist:
        print(idx, l)
        idx += 1
    print()

## Pick a project

In [ ]:
po = molpx._bmutils.path_object(available_fnames[0], basedir=basedir)

## Let molpx show it to you

In [ ]:
# Select what you want
#ptype = 'min_disp'
ptype = 'min_rmsd'
proj_idx = 1

lines = ['Sampling TIC_%u@%u frames for project %s. Change...'%(proj_idx, po.lagtime, po.fname.split('.')[0]),         
         ' * proj_idx to switch between TICs',
         ' * ptype to switch between path types', 
         ' * n_smooth parameter in visualize.sample', 
         ' * n_feats parameter in visualize.correlations', 
         '...or entirely to a different project', 
         'And of course, click anywhere on the FES or the slidebar!']
molpx.visualize._plot_ND_FES(np.vstack(po.data)[:,[0,1]], ['TIC_0', 'TIC_1'])
geom, sample = po.path(ptype, proj_idx)

# This is to get a good orientation...


iwd = molpx.visualize.sample(sample, geom, 
                              plt.gca(), 
                              clear_lines=False,
                              #superpose='backbone',
                             #superpose=False,
                              plot_path=True,                                   
                              n_smooth=0)

__, iwd = molpx.visualize.correlations(po.feature_TIC_correlation, 
                                   widget=iwd,
                                   proj_color_list=['green'],
                                   proj_idxs=proj_idx, 
                                   n_feats = 5,
                                   featurizer=po.feat)

idxs = geom.top.select('name CA')
iwd.add_ball_and_stick(selection=idxs)
iwd.center_view()
for line in lines:
    display(Text(line, disabled=True, width='400px'))
iwd